In [8]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from datetime import datetime
import uuid

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)  # Añadir pausa después de aceptar cookies

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles
max_inmuebles = 3050  # Número máximo de inmuebles que queremos recolectar

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble():
    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')
        url_actual = browser.current_url
        # Extraer datos del inmueble
        try:
            # Descripción del inmueble (H1)
            descripcion = soup_inmueble.find('h1').text.strip() if soup_inmueble.find('h1') else "Descripción no disponible"

            # Localización en el primer <p> después del <h1> (ejemplo: Centro (Galapagar))
            localizacion = soup_inmueble.find('p').text.strip() if soup_inmueble.find('p') else "Localización no disponible"

            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Agregar el timestamp de scrapeo y un identificador único
            data = {
                'id': str(uuid.uuid4()),  # ID único para cada inmueble
                'Descripción': descripcion,
                'Localización': localizacion,
                'Enlace': url_actual,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler',
                'timestamp_scrapeo': datetime.now().isoformat()  # Timestamp de scrapeo
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Función para avanzar al siguiente inmueble
def ir_al_siguiente_inmueble():
    try:
        siguiente_boton = WebDriverWait(browser, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.navigation__link.navigation__link--next'))
        )
        siguiente_boton.click()
        time.sleep(2)  # Pausa para cargar el siguiente inmueble
        return True
    except Exception as e:
        print(f"No se encontró el botón de 'Siguiente': {e}")
        return False

# Iniciar en el primer inmueble
try:
    primer_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.ad-preview'))
    )
    primer_inmueble.click()
    time.sleep(2)  # Pausa para cargar la página del primer inmueble
except Exception as e:
    print(f"Error al abrir el primer inmueble: {e}")

# Bucle para extraer datos de cada inmueble hasta alcanzar el máximo o agotar los inmuebles
while len(all_data) < max_inmuebles:
    data = extraer_datos_inmueble()
    if data:
        all_data.append(data)
        print(f"Inmueble {inmueble_counter}: {data}")
        inmueble_counter += 1

    # Intentar ir al siguiente inmueble
    if not ir_al_siguiente_inmueble():
        break  # Si no hay más inmuebles, salir del bucle

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('inmuebles_alquiler.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'id': 'd5c4c62a-f422-430a-bb28-ffd2003bde80', 'Descripción': 'Casa adosada en alquiler en Sector B', 'Localización': 'Sector B (Boadilla del Monte)', 'Enlace': 'https://www.pisos.com/alquilar/casa_adosada-sector_b28660-44200537282_100200/', 'Precio': '3.600', 'Superficie Construida': '448', 'Última Actualización': 'Anuncio actualizado el 21/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '448 m²'), ('Habitaciones:', '5'), ('Baños:', '2'), ('Tipo de casa:', 'Adosada'), ('Antigüedad:', 'Menos de 5 años'), ('Conservación:', 'En buen estado'), ('Referencia:', 'TC170-581681')], 'Tipo de operación': 'Alquiler', 'timestamp_scrapeo': '2024-11-04T11:42:25.912524'}
Inmueble 2: {'id': '4447b365-9ad9-40e7-8a57-8019611cf4ce', 'Descripción': 'Piso en alquiler en Calle de Vista Alegre', 'Localización': 'Somosaguas-Húmera-Los Ángeles (Pozuelo de Alarcón)', 'Enlace': 'https://www.pisos.com/alquilar/piso-somosaguas_humera_los_ang

In [9]:
df

,id,Descripción,Localización,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación,timestamp_scrapeo
0,d5c4c62a-f422-430a-bb28-ffd2003bde80,Casa adosada en alquiler en Sector B,Sector B (Boadilla del Monte),https://www.pisos.com/alquilar/casa_adosada-se...,3.600,448,Anuncio actualizado el 21/10/2024,N/A,N/A,"[(Superficie construida:, 448 m²), (Habitacion...",Alquiler,2024-11-04T11:42:25.912524
1,4447b365-9ad9-40e7-8a57-8019611cf4ce,Piso en alquiler en Calle de Vista Alegre,Somosaguas-Húmera-Los Ángeles (Pozuelo de Alar...,https://www.pisos.com/alquilar/piso-somosaguas...,1.450,69,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 69 m²), (Superficie ...",Alquiler,2024-11-04T11:42:34.398206
2,2cd188dc-43db-4bb8-87ea-15e72ec27b60,Casa unifamiliar en alquiler en Calle XIX,Las Matas-Los Peñascales (Las Rozas de Madrid),https://www.pisos.com/alquilar/casa_unifamilia...,4.000,600,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 600 m²), (Superficie...",Alquiler,2024-11-04T11:42:40.242927
3,1ccbb1b1-7a79-4c70-85f2-354dc10b354a,Piso en alquiler en Zona Avenida de Europa,Zona Avenida de Europa (Pozuelo de Alarcón),https://www.pisos.com/alquilar/piso-zona_aveni...,1.400,65,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 65 m²), (Habitacione...",Alquiler,2024-11-04T11:42:45.825335
4,18c4c00d-98f9-40dd-be9e-3a012b63d9b1,Chalet en alquiler en Núcleo Urbano,Núcleo Urbano (Villanueva de la Cañada),https://www.pisos.com/alquilar/chalet-villanue...,2.700,329,Anuncio actualizado el 28/10/2024,N/A,N/A,"[(Superficie construida:, 329 m²), (Superficie...",Alquiler,2024-11-04T11:42:51.662813
...,...,...,...,...,...,...,...,...,...,...,...,...
1981,cb7da359-204c-415f-999a-75b979b35d61,Apartamento en alquiler en Avenida de Montes d...,Ciudalcampo-Fuente del Fresno (San Sebastián d...,https://www.pisos.com/alquilar/apartamento-ciu...,1.700,101,Anuncio actualizado el 04/11/2024,Consumo:126 kWh/m² año,Emisiones:12 Kg CO₂/m² año,"[(Superficie construida:, 101 m²), (Habitacion...",Alquiler,2024-11-04T15:35:40.393501
1982,8e56eaf2-5c4c-426c-8501-e9114045f3e9,Piso en alquiler en Calle de Sagasta,Justicia-Chueca (Distrito Centro. Madrid Capital),https://www.pisos.com/alquilar/piso-justicia_c...,4.100,110,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 110 m²), (Superficie...",Alquiler,2024-11-04T15:35:47.964623
1983,882079aa-e895-4a7a-82eb-5736a3386069,Loft en alquiler en Calle del Cerro del Murmullo,Ensanche de Vallecas-Valdecarros (Distrito Vil...,https://www.pisos.com/alquilar/loft-ensanche_d...,900,49,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 49 m²), (Superficie ...",Alquiler,2024-11-04T15:35:55.661331
1984,c67c8dd7-b917-45ac-9702-d3a3253fc725,Piso en alquiler en Meco,Meco,https://www.pisos.com/alquilar/piso-meco_centr...,750,47,Anuncio actualizado el 04/11/2024,N/A,N/A,"[(Superficie construida:, 47 m²), (Habitacione...",Alquiler,2024-11-04T15:36:02.652517
